In [6]:
# main_app_safe.py - COMPLETE FIXED VERSION WITH IMPROVED JSON RECOMMENDATIONS
import os, re, json, traceback, csv
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv
from difflib import SequenceMatcher
import time
import threading
from datetime import datetime

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")

DATA_PATHS = [
    "symptom_list_with_recommendations_fixed.csv",
    "symptom_list_with_recommendations.csv",
    "symptom_list.csv",
    "symptom_list.xlsx",
]

RECOMMENDATIONS_JSON = "recommendations.json"
PHONE_LOG_FILE = "phone_callbacks.csv"

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- EMERGENCY KEYWORDS ----------
EMERGENCY_KEYWORDS = [
    "chest pain", "breathless", "shortness of breath", "difficulty breathing",
    "bleeding", "vomit blood", "blood in stool", "severe", "collapse",
    "unconscious", "faint", "loss of consciousness", "severe pain", "emergency"
]

# ---------- LOAD RECOMMENDATIONS JSON ----------
recommendations_data = None

def load_recommendations_json():
    """Load recommendations from JSON file with extensive error checking"""
    global recommendations_data
    
    print(f"\n{'='*60}")
    print(f"LOADING RECOMMENDATIONS JSON")
    print(f"{'='*60}")
    print(f"Looking for file: {RECOMMENDATIONS_JSON}")
    print(f"Current directory: {os.getcwd()}")
    print(f"File exists: {os.path.exists(RECOMMENDATIONS_JSON)}")
    
    if not os.path.exists(RECOMMENDATIONS_JSON):
        print(f"⚠️  Warning: {RECOMMENDATIONS_JSON} not found in {os.getcwd()}")
        # Try to find it in parent directories
        for parent in ['../', '../../']:
            alt_path = os.path.join(parent, RECOMMENDATIONS_JSON)
            if os.path.exists(alt_path):
                print(f"Found file at: {alt_path}")
                with open(alt_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    print(f"✓ Loaded {len(data.get('diseases', []))} disease recommendations")
                    return data
        print("⚠️  Using fallback recommendations.")
        return None
    
    try:
        with open(RECOMMENDATIONS_JSON, 'r', encoding='utf-8') as f:
            data = json.load(f)
            diseases = data.get('diseases', [])
            print(f"✓ Successfully loaded JSON")
            print(f"  Number of diseases: {len(diseases)}")
            if diseases:
                print(f"  First disease: {diseases[0].get('name', 'N/A')}")
                print(f"  Sample aliases: {diseases[0].get('aliases', [])}")
            print(f"{'='*60}\n")
            return data
    except json.JSONDecodeError as e:
        print(f"✗ JSON parsing error: {e}")
        print(f"  Line {e.lineno}, Column {e.colno}")
        return None
    except Exception as e:
        print(f"✗ Error loading recommendations JSON: {e}")
        traceback.print_exc()
        return None

recommendations_data = load_recommendations_json()

# ---------- UTIL ----------
def normalize_text(s):
    """Aggressive normalization for matching"""
    if not s:
        return ""
    # Convert to lowercase, remove special chars, normalize spaces
    text = str(s).lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Replace punctuation with space
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

def token_set(s):
    return set(t for t in normalize_text(s).split() if t and len(t) > 1)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

def similarity(a, b):
    """Calculate similarity between two strings"""
    return SequenceMatcher(None, normalize_text(a), normalize_text(b)).ratio()

def token_overlap_score(a, b):
    """Calculate token overlap score"""
    tokens_a = token_set(a)
    tokens_b = token_set(b)
    if not tokens_a or not tokens_b:
        return 0.0
    intersection = len(tokens_a & tokens_b)
    union = len(tokens_a | tokens_b)
    return intersection / union if union > 0 else 0.0

def detect_emergency(text):
    """Detect emergency keywords"""
    if not text:
        return False
    t = str(text).lower()
    for kw in EMERGENCY_KEYWORDS:
        if kw in t:
            return True
    return False

# ---------- LOAD CSV DATA ----------
_lock = threading.Lock()
_cache = {"df": None, "loaded_at": None, "path": None}

def _find_existing_path():
    for p in DATA_PATHS:
        if os.path.exists(p):
            return p
    return None

def _normalize_colname_map(cols):
    return { " ".join(str(c).strip().split()).lower(): c for c in cols }

def load_symptom_df():
    with _lock:
        if _cache["df"] is not None:
            return _cache["df"], _cache["path"]

        found = _find_existing_path()
        if not found:
            raise FileNotFoundError(f"No symptom file found. Searched: {DATA_PATHS}")

        try:
            if found.lower().endswith(".csv"):
                encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                df = None
                for encoding in encodings_to_try:
                    try:
                        df = pd.read_csv(found, encoding=encoding)
                        print(f"✓ Loaded CSV with encoding: {encoding}")
                        break
                    except UnicodeDecodeError:
                        continue
                if df is None:
                    raise RuntimeError("Failed to read CSV with any encoding")
            else:
                df = pd.read_excel(found)
        except Exception as e:
            raise RuntimeError(f"Failed to read symptom file {found}: {e}")

        # Normalize columns
        new_cols = [" ".join(str(c).strip().split()) for c in df.columns]
        df.columns = new_cols

        # Auto-detect and rename columns
        preferred_names = {
            "disease": ["disease", "disease name", "name"],
            "typical_symptoms": ["typical symptoms", "typical symptom", "symptoms"],
            "confirm_symptom": ["confirm symptom", "confirm symptoms", "confirm"],
            "unique_symptom": ["unique symptom", "unique symptoms"],
        }

        lowered_map = _normalize_colname_map(df.columns)

        def pick_column(candidate_list):
            for cand in candidate_list:
                if cand in lowered_map:
                    return lowered_map[cand]
            return None

        col_selected = {}
        for canon, cand_list in preferred_names.items():
            col_selected[canon] = pick_column(cand_list)

        friendly_rename = {}
        if col_selected.get("disease"):
            friendly_rename[col_selected["disease"]] = "Disease"
        if col_selected.get("typical_symptoms"):
            friendly_rename[col_selected["typical_symptoms"]] = "Typical Symptoms"
        if col_selected.get("confirm_symptom"):
            friendly_rename[col_selected["confirm_symptom"]] = "Confirm Symptom"
        if col_selected.get("unique_symptom"):
            friendly_rename[col_selected["unique_symptom"]] = "Unique Symptom"

        if friendly_rename:
            df = df.rename(columns=friendly_rename)

        for c in ["Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom"]:
            if c not in df.columns:
                df[c] = ""

        _cache["df"] = df
        _cache["loaded_at"] = time.time()
        _cache["path"] = found
        
        print(f"✓ CSV loaded successfully: {df.shape[0]} diseases")
        return df, found

try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s — shape %s", data_file_used, df.shape)
except Exception as e:
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    df = None

# ---------- JSON-BASED RECOMMENDATION FINDER (IMPROVED) ----------

def find_recommendation_json(disease_name):
    """
    IMPROVED: Find recommendation with better matching algorithm
    """
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("⚠️  No recommendations data available")
        return None
    
    disease_norm = normalize_text(disease_name)
    disease_tokens = token_set(disease_name)
    
    print(f"\n{'='*60}")
    print(f"SEARCHING FOR RECOMMENDATION")
    print(f"{'='*60}")
    print(f"Original: '{disease_name}'")
    print(f"Normalized: '{disease_norm}'")
    print(f"Tokens: {disease_tokens}")
    
    best_match = None
    best_score = 0.0
    best_method = None
    
    for idx, disease_entry in enumerate(recommendations_data['diseases']):
        entry_name = disease_entry.get('name', '')
        entry_name_norm = normalize_text(entry_name)
        
        if idx < 3:  # Show first few checks
            print(f"\n[{idx+1}] Checking: {entry_name}")
            print(f"    Normalized: {entry_name_norm}")
        
        # METHOD 1: EXACT MATCH (normalized)
        if disease_norm == entry_name_norm:
            print(f"  ✅✅✅ EXACT MATCH!")
            return disease_entry
        
        # METHOD 2: EXACT SUBSTRING MATCH
        if disease_norm in entry_name_norm or entry_name_norm in disease_norm:
            print(f"  ✅✅ SUBSTRING MATCH!")
            return disease_entry
        
        # METHOD 3: Token overlap
        entry_tokens = token_set(entry_name)
        overlap = len(disease_tokens & entry_tokens)
        overlap_score = overlap / max(len(disease_tokens), len(entry_tokens)) if disease_tokens or entry_tokens else 0
        
        if overlap_score > 0.6 and overlap >= 2:
            if idx < 3:
                print(f"  ✅ TOKEN OVERLAP MATCH (score: {overlap_score:.2f}, overlap: {overlap})")
            if overlap_score > best_score:
                best_score = overlap_score
                best_match = disease_entry
                best_method = "token_overlap"
        
        # METHOD 4: Sequence similarity
        sim_score = similarity(disease_name, entry_name)
        if sim_score > best_score:
            best_score = sim_score
            best_match = disease_entry
            best_method = "similarity"
            if idx < 3:
                print(f"  ~ Better similarity: {sim_score:.2f}")
        
        # METHOD 5: Check aliases
        for alias in disease_entry.get('aliases', []):
            alias_norm = normalize_text(alias)
            
            # Exact alias match
            if disease_norm == alias_norm:
                print(f"  ✅✅✅ EXACT ALIAS MATCH: '{alias}'")
                return disease_entry
            
            # Substring in alias
            if disease_norm in alias_norm or alias_norm in disease_norm:
                print(f"  ✅✅ ALIAS SUBSTRING MATCH: '{alias}'")
                return disease_entry
            
            # Alias similarity
            alias_sim = similarity(disease_name, alias)
            if alias_sim > best_score:
                best_score = alias_sim
                best_match = disease_entry
                best_method = f"alias_similarity:{alias}"
    
    # Return best match if good enough
    if best_match and best_score >= 0.50:  # Lowered threshold
        print(f"\n✅ FUZZY MATCH: {best_match.get('name')}")
        print(f"  Method: {best_method}")
        print(f"  Score: {best_score:.2f}")
        return best_match
    
    print(f"\n❌ NO MATCH FOUND")
    print(f"  Best score: {best_score:.2f}")
    print(f"  Best candidate: {best_match.get('name') if best_match else 'None'}")
    return None

def format_recommendation_html_from_json(recommendation):
    """Format JSON recommendation into HTML"""
    if not recommendation:
        return "<div class='small-muted'>No recommendations available.</div>"
    
    medications = recommendation.get('medications', '').strip()
    home_remedies = recommendation.get('home_remedies', '').strip()
    
    html_parts = []
    
    # Check for urgent conditions
    urgent_keywords = ['seek clinician', 'consult a doctor', 'please consult', 'medical attention', 'no otc']
    is_urgent = any(kw in medications.lower() for kw in urgent_keywords)
    
    if medications:
        if is_urgent:
            html_parts.append(
                f"<div class='urgent-warning'>"
                f"<strong>⚠️ Medical Attention Required</strong>"
                f"<div style='margin-top:8px;color:#475569;'>{medications}</div>"
                f"</div>"
            )
        elif medications.lower() != 'please consult a doctor':
            html_parts.append(
                f"<div class='medication'>"
                f"<strong>💊 Medications</strong>"
                f"<div style='margin-top:6px;'>{medications}</div>"
                f"</div>"
            )
    
    if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
        html_parts.append(
            f"<div class='home-remedy'>"
            f"<strong>🏡 Home Remedies</strong>"
            f"<div style='margin-top:6px;'>{home_remedies}</div>"
            f"</div>"
        )
    
    if not html_parts:
        html_parts.append(
            "<div class='small-muted' style='padding:10px;'>"
            "Please rest, stay hydrated, and consult a healthcare professional if symptoms persist."
            "</div>"
        )
    
    return "".join(html_parts)

# ---------- ENDPOINTS ----------

@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500
    return jsonify({
        "file": data_file_used,
        "shape": list(df.shape),
        "columns": df.columns.tolist(),
        "sample_first_rows": df.head(5).to_dict(orient="records"),
        "recommendations_loaded": recommendations_data is not None,
        "num_recommendations": len(recommendations_data.get('diseases', [])) if recommendations_data else 0
    })

@app.route("/_test_json_load", methods=["GET"])
def test_json_load():
    """Test if JSON is properly loaded"""
    return jsonify({
        "json_file_path": RECOMMENDATIONS_JSON,
        "file_exists": os.path.exists(RECOMMENDATIONS_JSON),
        "json_loaded": recommendations_data is not None,
        "num_diseases": len(recommendations_data.get('diseases', [])) if recommendations_data else 0,
        "first_5_diseases": [
            d.get('name') for d in recommendations_data.get('diseases', [])[:5]
        ] if recommendations_data else [],
        "current_directory": os.getcwd(),
        "directory_contents": os.listdir('.')[:20]
    })

@app.route("/_test_match/<path:disease_name>", methods=["GET"])
def test_match(disease_name):
    """Test matching for a specific disease name"""
    print(f"\n{'='*60}")
    print(f"MANUAL TEST FOR: {disease_name}")
    print(f"{'='*60}")
    
    result = find_recommendation_json(disease_name)
    
    if result:
        return jsonify({
            "success": True,
            "searched_for": disease_name,
            "matched_disease": result.get('name'),
            "medications": result.get('medications'),
            "home_remedies": result.get('home_remedies'),
            "full_data": result
        })
    else:
        # Try to find close matches
        close_matches = []
        if recommendations_data and 'diseases' in recommendations_data:
            for d in recommendations_data['diseases'][:20]:
                score = similarity(disease_name, d.get('name', ''))
                if score > 0.3:
                    close_matches.append({
                        "name": d.get('name'),
                        "score": round(score, 3)
                    })
        
        return jsonify({
            "success": False,
            "searched_for": disease_name,
            "normalized": normalize_text(disease_name),
            "close_matches": sorted(close_matches, key=lambda x: x['score'], reverse=True)[:5]
        })

@app.route("/_list_all_diseases", methods=["GET"])
def list_all_diseases():
    """List all diseases in JSON for reference"""
    if not recommendations_data or 'diseases' not in recommendations_data:
        return jsonify({"error": "No data loaded"}), 500
    
    diseases = []
    for d in recommendations_data['diseases']:
        diseases.append({
            "name": d.get('name'),
            "normalized": normalize_text(d.get('name', '')),
            "aliases": d.get('aliases', []),
            "has_medications": bool(d.get('medications')),
            "has_home_remedies": bool(d.get('home_remedies'))
        })
    
    return jsonify({
        "total": len(diseases),
        "diseases": diseases
    })

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User: {user_text}. Possible: {[p['disease'] for p in possible]}. Ask about: {confirm_list}"}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("No OpenAI key")
    except Exception:
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    reply_text = str(user_reply).lower()
    reply_tokens = set(reply_text.split())
    
    explicit_yes = any(word in reply_tokens for word in ['yes', 'yeah', 'yep', 'correct', 'right'])
    explicit_no = any(word in reply_tokens for word in ['no', 'nope', 'not', 'none'])
    
    best_match = None
    best_disease_name = None
    best_count = 0
    
    for p in possible:
        disease_name = p.get("disease", "Unknown")
        confirm_symptoms = p.get("confirm_symptoms", [])
        
        count = sum(1 for sym in confirm_symptoms if str(sym).lower().replace("-", " ") in reply_text)
        
        if count > best_count:
            best_count = count
            best_match = disease_name
            best_disease_name = disease_name

    if explicit_yes and not explicit_no and possible:
        best_disease_name = possible[0].get("disease", "Unknown")
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": 0.85
        })
    elif explicit_no:
        return jsonify({
            "stage": "final_result",
            "disease": "Symptoms unclear",
            "confidence": 0.0
        })
    elif best_match and best_count > 0:
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": min(0.9, 0.5 + (best_count * 0.1))
        })
    else:
        fallback = possible[0].get("disease", "Unknown") if possible else "Unclear"
        return jsonify({
            "stage": "final_result",
            "disease": fallback,
            "confidence": 0.3
        })

@app.route("/recommend", methods=["POST"])
def recommend():
    """JSON-based recommendation endpoint with improved matching"""
    
    print("\n" + "="*80)
    print("RECOMMEND ENDPOINT CALLED")
    print("="*80)
    
    payload = request.get_json(silent=True) or {}
    disease = payload.get("disease", "").strip()
    tokens = payload.get("tokens", [])

    print(f"Raw payload: {payload}")
    print(f"Requested disease: '{disease}'")
    print(f"Tokens: {tokens[:10] if tokens else 'none'}")

    if not disease:
        print("❌ No disease name provided")
        return jsonify({
            "ok": False,
            "advice_text": "No disease specified",
            "advice_html": "<div class='small-muted'>Unable to provide recommendations.</div>",
            "urgent": False
        })

    # Check if JSON is loaded
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("❌ Recommendations data not loaded!")
        return jsonify({
            "ok": False,
            "advice_text": "Recommendation database not available",
            "advice_html": "<div class='small-muted'>Please rest, stay hydrated, and consult a doctor.</div>",
            "urgent": False
        })

    print(f"Searching through {len(recommendations_data['diseases'])} diseases...")

    # Find matching recommendation
    recommendation = find_recommendation_json(disease)
    
    if recommendation:
        print(f"✅ MATCH FOUND: {recommendation.get('name')}")
        
        empathetic_msg = recommendation.get('empathetic_message', '')
        medications = recommendation.get('medications', '')
        home_remedies = recommendation.get('home_remedies', '')
        
        # Create HTML
        html = format_recommendation_html_from_json(recommendation)
        
        # Build speech text
        speech_parts = []
        if medications and 'consult' not in medications.lower():
            speech_parts.append(f"You can take {medications}")
        if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
            speech_parts.append(f"Try these home remedies: {home_remedies}")
        
        speech_text = ". ".join(speech_parts) if speech_parts else "Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        
        urgent = detect_emergency(medications + ' ' + home_remedies) or 'seek' in medications.lower() or 'consult' in medications.lower()
        
        print(f"✓ Returning recommendation")
        print(f"  Medications: {medications[:50]}...")
        print(f"  Home remedies: {home_remedies[:50]}...")
        print(f"  Urgent: {urgent}")
        
        return jsonify({
            "ok": True,
            "matched_disease": recommendation.get('name'),
            "advice_text": f"Medications: {medications}; Home remedies: {home_remedies}",
            "advice_html": html,
            "advice_text_for_speech": speech_text,
            "full_message": empathetic_msg,
            "urgent": urgent,
            "source": "json_database"
        })
    else:
        # Fallback
        print("❌ NO MATCH FOUND - Using fallback")
        print(f"  Searched for: '{disease}'")
        print(f"  Normalized: '{normalize_text(disease)}'")
        
        generic = "I'm sorry, I couldn't find specific recommendations for this condition. Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        return jsonify({
            "ok": True,
            "matched_disease": None,
            "advice_text": generic,
            "advice_html": f"<div class='small-muted'>{generic}</div>",
            "advice_text_for_speech": generic,
            "urgent": False,
            "source": "fallback"
        })

@app.route("/save_phone", methods=["POST"])
def save_phone():
    try:
        payload = request.get_json(silent=True) or {}
        phone = payload.get("phone", "").strip()
        disease = payload.get("disease", "Unknown")
        confidence = payload.get("confidence", 0)
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone required"}), 400
        
        digits = re.sub(r'[^\d]', '', phone)
        if not (10 <= len(digits) <= 14):
            return jsonify({"ok": False, "message": "Invalid phone format"}), 400
        
        file_exists = os.path.exists(PHONE_LOG_FILE)
        with open(PHONE_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['Timestamp', 'Phone', 'Disease', 'Confidence', 'Status'])
            writer.writerow([datetime.now().strftime('%Y-%m-%d %H:%M:%S'), phone, disease, confidence, 'Pending'])
        
        return jsonify({
            "ok": True,
            "message": "Phone saved successfully",
            "phone": phone
        })
    except Exception as e:
        return jsonify({"ok": False, "message": str(e)}), 500

if __name__ == "__main__":
    print("\n" + "="*60)
    print("STARTUP CHECK")
    print("="*60)
    print(f"CSV loaded: {df is not None}")
    if df is not None:
        print(f"CSV shape: {df.shape}")
        print(f"CSV columns: {df.columns.tolist()}")
    print(f"Recommendations JSON loaded: {recommendations_data is not None}")
    if recommendations_data:
        print(f"Number of diseases in JSON: {len(recommendations_data.get('diseases', []))}")
        print(f"First 3 diseases: {[d.get('name') for d in recommendations_data.get('diseases', [])[:3]]}")
    print("="*60 + "\n")
    
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


# ============================================
# DOCTOR DASHBOARD ENDPOINTS
# ============================================

@app.route("/get_patients", methods=["GET"])
def get_patients():
    """
    Get all patients from the phone callbacks CSV file for doctor dashboard
    Handles both old format (without Name) and new format (with Name)
    """
    try:
        patients = []
        
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({
                "ok": True,
                "patients": [],
                "message": "No patients yet"
            })
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for idx, row in enumerate(reader, 1):
                # Handle both old and new CSV formats
                name = row.get('Name', None)
                if not name or name.strip() == '':
                    # Generate a default name if missing
                    name = f'Patient #{idx}'
                
                patients.append({
                    "timestamp": row.get('Timestamp', ''),
                    "name": name,
                    "phone": row.get('Phone', ''),
                    "disease": row.get('Disease', 'Unknown'),
                    "confidence": int(row.get('Confidence', 0)),
                    "status": row.get('Status', 'Pending')
                })
        
        # Sort by timestamp (most recent first)
        patients.sort(key=lambda x: x['timestamp'], reverse=True)
        
        print(f"📋 Returning {len(patients)} patients")
        
        return jsonify({
            "ok": True,
            "patients": patients,
            "total": len(patients)
        })
        
    except Exception as e:
        print(f"❌ Error getting patients: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e),
            "patients": []
        }), 500


@app.route("/update_patient_status", methods=["POST"])
def update_patient_status():
    """
    Update the status of a patient (e.g., Pending -> Called -> Completed)
    """
    try:
        data = request.get_json(silent=True) or {}
        phone = data.get('phone', '').strip()
        new_status = data.get('status', 'Called').strip()
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone number required"}), 400
        
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({"ok": False, "message": "No patient records found"}), 404
        
        # Read all rows
        rows = []
        updated = False
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            fieldnames = reader.fieldnames
            for row in reader:
                if row['Phone'] == phone:
                    row['Status'] = new_status
                    updated = True
                    print(f"✅ Updated {phone} to {new_status}")
                rows.append(row)
        
        if not updated:
            return jsonify({"ok": False, "message": "Patient not found"}), 404
        
        # Write back
        with open(PHONE_LOG_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
        
        return jsonify({
            "ok": True,
            "message": f"Status updated to {new_status}",
            "phone": phone,
            "status": new_status
        })
        
    except Exception as e:
        print(f"❌ Error updating patient status: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500


@app.route("/get_patient_details/<phone>", methods=["GET"])
def get_patient_details(phone):
    """
    Get detailed information for a specific patient
    """
    try:
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({"ok": False, "message": "No patient records"}), 404
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row.get('Phone') == phone:
                    return jsonify({
                        "ok": True,
                        "patient": {
                            "timestamp": row.get('Timestamp', ''),
                            "name": row.get('Name', 'Unknown'),
                            "phone": row.get('Phone', ''),
                            "disease": row.get('Disease', 'Unknown'),
                            "confidence": int(row.get('Confidence', 0)),
                            "status": row.get('Status', 'Pending')
                        }
                    })
        
        return jsonify({
            "ok": False,
            "message": "Patient not found"
        }), 404
        
    except Exception as e:
        print(f"❌ Error getting patient details: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500


@app.route("/get_dashboard_stats", methods=["GET"])
def get_dashboard_stats():
    """
    Get statistics for doctor dashboard
    """
    try:
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({
                "ok": True,
                "stats": {
                    "total": 0,
                    "pending": 0,
                    "called": 0,
                    "completed": 0,
                    "today": 0,
                    "urgent": 0
                }
            })
        
        total = 0
        pending = 0
        called = 0
        completed = 0
        today = 0
        urgent = 0
        
        today_date = datetime.now().date()
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                total += 1
                status = row.get('Status', 'Pending')
                
                if status == 'Pending':
                    pending += 1
                elif status == 'Called':
                    called += 1
                elif status == 'Completed':
                    completed += 1
                
                # Check if today
                timestamp_str = row.get('Timestamp', '')
                if timestamp_str:
                    try:
                        record_date = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S').date()
                        if record_date == today_date:
                            today += 1
                    except:
                        pass
                
                # Check if urgent (low confidence or serious disease)
                confidence = int(row.get('Confidence', 100))
                disease = row.get('Disease', '').lower()
                urgent_keywords = ['pneumonia', 'heart', 'stroke', 'breathing', 'chest']
                
                if confidence < 50 or any(kw in disease for kw in urgent_keywords):
                    urgent += 1
        
        return jsonify({
            "ok": True,
            "stats": {
                "total": total,
                "pending": pending,
                "called": called,
                "completed": completed,
                "today": today,
                "urgent": urgent
            }
        })
        
    except Exception as e:
        print(f"Error getting dashboard stats: {e}")
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500

@app.route("/save_phone", methods=["POST"])
def save_phone():
    try:
        payload = request.get_json(silent=True) or {}
        phone = payload.get("phone", "").strip()
        name = payload.get("name", "").strip()  # Optional now
        disease = payload.get("disease", "Unknown")
        confidence = payload.get("confidence", 0)
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone required"}), 400
        
        digits = re.sub(r'[^\d]', '', phone)
        if not (10 <= len(digits) <= 14):
            return jsonify({"ok": False, "message": "Invalid phone format"}), 400
        
        # Check if file exists and has Name column
        file_exists = os.path.exists(PHONE_LOG_FILE)
        has_name_column = False
        
        if file_exists:
            with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
                first_line = f.readline().strip()
                has_name_column = 'Name' in first_line
        
        with open(PHONE_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # Write header if file is new
            if not file_exists:
                writer.writerow(['Timestamp', 'Name', 'Phone', 'Disease', 'Confidence', 'Status'])
                has_name_column = True
            
            # Write data row
            if has_name_column:
                writer.writerow([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    name or 'Unknown Patient',
                    phone,
                    disease,
                    confidence,
                    'Pending'
                ])
            else:
                # Old format without Name column
                writer.writerow([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    phone,
                    disease,
                    confidence,
                    'Pending'
                ])
        
        print(f"✅ Saved patient: {name or 'Unknown'} ({phone}) - {disease}")
        
        return jsonify({
            "ok": True,
            "message": "Details saved successfully",
            "name": name,
            "phone": phone
        })
    except Exception as e:
        print(f"❌ Error saving patient: {e}")
        traceback.print_exc()
        return jsonify({"ok": False, "message": str(e)}), 500



LOADING RECOMMENDATIONS JSON
Looking for file: recommendations.json
Current directory: c:\Users\Jay\Desktop\SE proj\New folder\Back
File exists: True
✓ Successfully loaded JSON
  Number of diseases: 62
  First disease: Common Cold (Upper Respiratory Tract Infection)
  Sample aliases: ['common cold (upper respiratory tract infection)']

✓ Loaded CSV with encoding: utf-8
✓ CSV loaded successfully: 62 diseases

STARTUP CHECK
CSV loaded: True
CSV shape: (62, 6)
CSV columns: ['Sr No.', 'Disease', 'Typical Symptoms', 'Unique Symptom', 'Confirm Symptom', 'Recommendations']
Recommendations JSON loaded: True
Number of diseases in JSON: 62
First 3 diseases: ['Common Cold (Upper Respiratory Tract Infection)', 'Influenza (Flu)', 'COVID-19']

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit


AssertionError: View function mapping is overwriting an existing endpoint function: save_phone

In [1]:
# main_app_safe.py - COMPLETE FIXED VERSION WITH IMPROVED JSON RECOMMENDATIONS
import os, re, json, traceback, csv
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv
from difflib import SequenceMatcher
import time
import threading
from datetime import datetime

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")

DATA_PATHS = [
    "symptom_list_with_recommendations_fixed.csv",
    "symptom_list_with_recommendations.csv",
    "symptom_list.csv",
    "symptom_list.xlsx",
]

RECOMMENDATIONS_JSON = "recommendations.json"
PHONE_LOG_FILE = "phone_callbacks.csv"

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- EMERGENCY KEYWORDS ----------
EMERGENCY_KEYWORDS = [
    "chest pain", "breathless", "shortness of breath", "difficulty breathing",
    "bleeding", "vomit blood", "blood in stool", "severe", "collapse",
    "unconscious", "faint", "loss of consciousness", "severe pain", "emergency"
]

# ---------- LOAD RECOMMENDATIONS JSON ----------
recommendations_data = None

def load_recommendations_json():
    """Load recommendations from JSON file with extensive error checking"""
    global recommendations_data
    
    print(f"\n{'='*60}")
    print(f"LOADING RECOMMENDATIONS JSON")
    print(f"{'='*60}")
    print(f"Looking for file: {RECOMMENDATIONS_JSON}")
    print(f"Current directory: {os.getcwd()}")
    print(f"File exists: {os.path.exists(RECOMMENDATIONS_JSON)}")
    
    if not os.path.exists(RECOMMENDATIONS_JSON):
        print(f"⚠️ Warning: {RECOMMENDATIONS_JSON} not found in {os.getcwd()}")
        # Try to find it in parent directories
        for parent in ['../', '../../']:
            alt_path = os.path.join(parent, RECOMMENDATIONS_JSON)
            if os.path.exists(alt_path):
                print(f"Found file at: {alt_path}")
                with open(alt_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    print(f"✓ Loaded {len(data.get('diseases', []))} disease recommendations")
                    return data
        print("⚠️ Using fallback recommendations.")
        return None
    
    try:
        with open(RECOMMENDATIONS_JSON, 'r', encoding='utf-8') as f:
            data = json.load(f)
            diseases = data.get('diseases', [])
            print(f"✓ Successfully loaded JSON")
            print(f"  Number of diseases: {len(diseases)}")
            if diseases:
                print(f"  First disease: {diseases[0].get('name', 'N/A')}")
                print(f"  Sample aliases: {diseases[0].get('aliases', [])}")
            print(f"{'='*60}\n")
            return data
    except json.JSONDecodeError as e:
        print(f"✗ JSON parsing error: {e}")
        print(f"  Line {e.lineno}, Column {e.colno}")
        return None
    except Exception as e:
        print(f"✗ Error loading recommendations JSON: {e}")
        traceback.print_exc()
        return None

recommendations_data = load_recommendations_json()

# ---------- UTIL ----------
def normalize_text(s):
    """Aggressive normalization for matching"""
    if not s:
        return ""
    # Convert to lowercase, remove special chars, normalize spaces
    text = str(s).lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Replace punctuation with space
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

def token_set(s):
    return set(t for t in normalize_text(s).split() if t and len(t) > 1)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

def similarity(a, b):
    """Calculate similarity between two strings"""
    return SequenceMatcher(None, normalize_text(a), normalize_text(b)).ratio()

def token_overlap_score(a, b):
    """Calculate token overlap score"""
    tokens_a = token_set(a)
    tokens_b = token_set(b)
    if not tokens_a or not tokens_b:
        return 0.0
    intersection = len(tokens_a & tokens_b)
    union = len(tokens_a | tokens_b)
    return intersection / union if union > 0 else 0.0

def detect_emergency(text):
    """Detect emergency keywords"""
    if not text:
        return False
    t = str(text).lower()
    for kw in EMERGENCY_KEYWORDS:
        if kw in t:
            return True
    return False

# ---------- LOAD CSV DATA ----------
_lock = threading.Lock()
_cache = {"df": None, "loaded_at": None, "path": None}

def _find_existing_path():
    for p in DATA_PATHS:
        if os.path.exists(p):
            return p
    return None

def _normalize_colname_map(cols):
    return { " ".join(str(c).strip().split()).lower(): c for c in cols }

def load_symptom_df():
    with _lock:
        if _cache["df"] is not None:
            return _cache["df"], _cache["path"]

        found = _find_existing_path()
        if not found:
            raise FileNotFoundError(f"No symptom file found. Searched: {DATA_PATHS}")

        try:
            if found.lower().endswith(".csv"):
                encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                df = None
                for encoding in encodings_to_try:
                    try:
                        df = pd.read_csv(found, encoding=encoding)
                        print(f"✓ Loaded CSV with encoding: {encoding}")
                        break
                    except UnicodeDecodeError:
                        continue
                if df is None:
                    raise RuntimeError("Failed to read CSV with any encoding")
            else:
                df = pd.read_excel(found)
        except Exception as e:
            raise RuntimeError(f"Failed to read symptom file {found}: {e}")

        # Normalize columns
        new_cols = [" ".join(str(c).strip().split()) for c in df.columns]
        df.columns = new_cols

        # Auto-detect and rename columns
        preferred_names = {
            "disease": ["disease", "disease name", "name"],
            "typical_symptoms": ["typical symptoms", "typical symptom", "symptoms"],
            "confirm_symptom": ["confirm symptom", "confirm symptoms", "confirm"],
            "unique_symptom": ["unique symptom", "unique symptoms"],
        }

        lowered_map = _normalize_colname_map(df.columns)

        def pick_column(candidate_list):
            for cand in candidate_list:
                if cand in lowered_map:
                    return lowered_map[cand]
            return None

        col_selected = {}
        for canon, cand_list in preferred_names.items():
            col_selected[canon] = pick_column(cand_list)

        friendly_rename = {}
        if col_selected.get("disease"):
            friendly_rename[col_selected["disease"]] = "Disease"
        if col_selected.get("typical_symptoms"):
            friendly_rename[col_selected["typical_symptoms"]] = "Typical Symptoms"
        if col_selected.get("confirm_symptom"):
            friendly_rename[col_selected["confirm_symptom"]] = "Confirm Symptom"
        if col_selected.get("unique_symptom"):
            friendly_rename[col_selected["unique_symptom"]] = "Unique Symptom"

        if friendly_rename:
            df = df.rename(columns=friendly_rename)

        for c in ["Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom"]:
            if c not in df.columns:
                df[c] = ""

        _cache["df"] = df
        _cache["loaded_at"] = time.time()
        _cache["path"] = found
        
        print(f"✓ CSV loaded successfully: {df.shape[0]} diseases")
        return df, found

try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s – shape %s", data_file_used, df.shape)
except Exception as e:
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    df = None

# ---------- JSON-BASED RECOMMENDATION FINDER (IMPROVED) ----------

def find_recommendation_json(disease_name):
    """
    IMPROVED: Find recommendation with better matching algorithm
    """
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("⚠️ No recommendations data available")
        return None
    
    disease_norm = normalize_text(disease_name)
    disease_tokens = token_set(disease_name)
    
    print(f"\n{'='*60}")
    print(f"SEARCHING FOR RECOMMENDATION")
    print(f"{'='*60}")
    print(f"Original: '{disease_name}'")
    print(f"Normalized: '{disease_norm}'")
    print(f"Tokens: {disease_tokens}")
    
    best_match = None
    best_score = 0.0
    best_method = None
    
    for idx, disease_entry in enumerate(recommendations_data['diseases']):
        entry_name = disease_entry.get('name', '')
        entry_name_norm = normalize_text(entry_name)
        
        if idx < 3:  # Show first few checks
            print(f"\n[{idx+1}] Checking: {entry_name}")
            print(f"    Normalized: {entry_name_norm}")
        
        # METHOD 1: EXACT MATCH (normalized)
        if disease_norm == entry_name_norm:
            print(f"  ✅✅✅ EXACT MATCH!")
            return disease_entry
        
        # METHOD 2: EXACT SUBSTRING MATCH
        if disease_norm in entry_name_norm or entry_name_norm in disease_norm:
            print(f"  ✅✅ SUBSTRING MATCH!")
            return disease_entry
        
        # METHOD 3: Token overlap
        entry_tokens = token_set(entry_name)
        overlap = len(disease_tokens & entry_tokens)
        overlap_score = overlap / max(len(disease_tokens), len(entry_tokens)) if disease_tokens or entry_tokens else 0
        
        if overlap_score > 0.6 and overlap >= 2:
            if idx < 3:
                print(f"  ✅ TOKEN OVERLAP MATCH (score: {overlap_score:.2f}, overlap: {overlap})")
            if overlap_score > best_score:
                best_score = overlap_score
                best_match = disease_entry
                best_method = "token_overlap"
        
        # METHOD 4: Sequence similarity
        sim_score = similarity(disease_name, entry_name)
        if sim_score > best_score:
            best_score = sim_score
            best_match = disease_entry
            best_method = "similarity"
            if idx < 3:
                print(f"  ~ Better similarity: {sim_score:.2f}")
        
        # METHOD 5: Check aliases
        for alias in disease_entry.get('aliases', []):
            alias_norm = normalize_text(alias)
            
            # Exact alias match
            if disease_norm == alias_norm:
                print(f"  ✅✅✅ EXACT ALIAS MATCH: '{alias}'")
                return disease_entry
            
            # Substring in alias
            if disease_norm in alias_norm or alias_norm in disease_norm:
                print(f"  ✅✅ ALIAS SUBSTRING MATCH: '{alias}'")
                return disease_entry
            
            # Alias similarity
            alias_sim = similarity(disease_name, alias)
            if alias_sim > best_score:
                best_score = alias_sim
                best_match = disease_entry
                best_method = f"alias_similarity:{alias}"
    
    # Return best match if good enough
    if best_match and best_score >= 0.50:  # Lowered threshold
        print(f"\n✅ FUZZY MATCH: {best_match.get('name')}")
        print(f"  Method: {best_method}")
        print(f"  Score: {best_score:.2f}")
        return best_match
    
    print(f"\n❌ NO MATCH FOUND")
    print(f"  Best score: {best_score:.2f}")
    print(f"  Best candidate: {best_match.get('name') if best_match else 'None'}")
    return None

def format_recommendation_html_from_json(recommendation):
    """Format JSON recommendation into HTML"""
    if not recommendation:
        return "<div class='small-muted'>No recommendations available.</div>"
    
    medications = recommendation.get('medications', '').strip()
    home_remedies = recommendation.get('home_remedies', '').strip()
    
    html_parts = []
    
    # Check for urgent conditions
    urgent_keywords = ['seek clinician', 'consult a doctor', 'please consult', 'medical attention', 'no otc']
    is_urgent = any(kw in medications.lower() for kw in urgent_keywords)
    
    if medications:
        if is_urgent:
            html_parts.append(
                f"<div class='urgent-warning'>"
                f"<strong>⚠️ Medical Attention Required</strong>"
                f"<div style='margin-top:8px;color:#475569;'>{medications}</div>"
                f"</div>"
            )
        elif medications.lower() != 'please consult a doctor':
            html_parts.append(
                f"<div class='medication'>"
                f"<strong>💊 Medications</strong>"
                f"<div style='margin-top:6px;'>{medications}</div>"
                f"</div>"
            )
    
    if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
        html_parts.append(
            f"<div class='home-remedy'>"
            f"<strong>🏡 Home Remedies</strong>"
            f"<div style='margin-top:6px;'>{home_remedies}</div>"
            f"</div>"
        )
    
    if not html_parts:
        html_parts.append(
            "<div class='small-muted' style='padding:10px;'>"
            "Please rest, stay hydrated, and consult a healthcare professional if symptoms persist."
            "</div>"
        )
    
    return "".join(html_parts)

# ---------- ENDPOINTS ----------

@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500
    return jsonify({
        "file": data_file_used,
        "shape": list(df.shape),
        "columns": df.columns.tolist(),
        "sample_first_rows": df.head(5).to_dict(orient="records"),
        "recommendations_loaded": recommendations_data is not None,
        "num_recommendations": len(recommendations_data.get('diseases', [])) if recommendations_data else 0
    })

@app.route("/_test_json_load", methods=["GET"])
def test_json_load():
    """Test if JSON is properly loaded"""
    return jsonify({
        "json_file_path": RECOMMENDATIONS_JSON,
        "file_exists": os.path.exists(RECOMMENDATIONS_JSON),
        "json_loaded": recommendations_data is not None,
        "num_diseases": len(recommendations_data.get('diseases', [])) if recommendations_data else 0,
        "first_5_diseases": [
            d.get('name') for d in recommendations_data.get('diseases', [])[:5]
        ] if recommendations_data else [],
        "current_directory": os.getcwd(),
        "directory_contents": os.listdir('.')[:20]
    })

@app.route("/_test_match/<path:disease_name>", methods=["GET"])
def test_match(disease_name):
    """Test matching for a specific disease name"""
    print(f"\n{'='*60}")
    print(f"MANUAL TEST FOR: {disease_name}")
    print(f"{'='*60}")
    
    result = find_recommendation_json(disease_name)
    
    if result:
        return jsonify({
            "success": True,
            "searched_for": disease_name,
            "matched_disease": result.get('name'),
            "medications": result.get('medications'),
            "home_remedies": result.get('home_remedies'),
            "full_data": result
        })
    else:
        # Try to find close matches
        close_matches = []
        if recommendations_data and 'diseases' in recommendations_data:
            for d in recommendations_data['diseases'][:20]:
                score = similarity(disease_name, d.get('name', ''))
                if score > 0.3:
                    close_matches.append({
                        "name": d.get('name'),
                        "score": round(score, 3)
                    })
        
        return jsonify({
            "success": False,
            "searched_for": disease_name,
            "normalized": normalize_text(disease_name),
            "close_matches": sorted(close_matches, key=lambda x: x['score'], reverse=True)[:5]
        })

@app.route("/_list_all_diseases", methods=["GET"])
def list_all_diseases():
    """List all diseases in JSON for reference"""
    if not recommendations_data or 'diseases' not in recommendations_data:
        return jsonify({"error": "No data loaded"}), 500
    
    diseases = []
    for d in recommendations_data['diseases']:
        diseases.append({
            "name": d.get('name'),
            "normalized": normalize_text(d.get('name', '')),
            "aliases": d.get('aliases', []),
            "has_medications": bool(d.get('medications')),
            "has_home_remedies": bool(d.get('home_remedies'))
        })
    
    return jsonify({
        "total": len(diseases),
        "diseases": diseases
    })

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User: {user_text}. Possible: {[p['disease'] for p in possible]}. Ask about: {confirm_list}"}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("No OpenAI key")
    except Exception:
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    reply_text = str(user_reply).lower()
    reply_tokens = set(reply_text.split())
    
    explicit_yes = any(word in reply_tokens for word in ['yes', 'yeah', 'yep', 'correct', 'right'])
    explicit_no = any(word in reply_tokens for word in ['no', 'nope', 'not', 'none'])
    
    best_match = None
    best_disease_name = None
    best_count = 0
    
    for p in possible:
        disease_name = p.get("disease", "Unknown")
        confirm_symptoms = p.get("confirm_symptoms", [])
        
        count = sum(1 for sym in confirm_symptoms if str(sym).lower().replace("-", " ") in reply_text)
        
        if count > best_count:
            best_count = count
            best_match = disease_name
            best_disease_name = disease_name

    if explicit_yes and not explicit_no and possible:
        best_disease_name = possible[0].get("disease", "Unknown")
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": 0.85
        })
    elif explicit_no:
        return jsonify({
            "stage": "final_result",
            "disease": "Symptoms unclear",
            "confidence": 0.0
        })
    elif best_match and best_count > 0:
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": min(0.9, 0.5 + (best_count * 0.1))
        })
    else:
        fallback = possible[0].get("disease", "Unknown") if possible else "Unclear"
        return jsonify({
            "stage": "final_result",
            "disease": fallback,
            "confidence": 0.3
        })

@app.route("/recommend", methods=["POST"])
def recommend():
    """JSON-based recommendation endpoint with improved matching"""
    
    print("\n" + "="*80)
    print("RECOMMEND ENDPOINT CALLED")
    print("="*80)
    
    payload = request.get_json(silent=True) or {}
    disease = payload.get("disease", "").strip()
    tokens = payload.get("tokens", [])

    print(f"Raw payload: {payload}")
    print(f"Requested disease: '{disease}'")
    print(f"Tokens: {tokens[:10] if tokens else 'none'}")

    if not disease:
        print("❌ No disease name provided")
        return jsonify({
            "ok": False,
            "advice_text": "No disease specified",
            "advice_html": "<div class='small-muted'>Unable to provide recommendations.</div>",
            "urgent": False
        })

    # Check if JSON is loaded
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("❌ Recommendations data not loaded!")
        return jsonify({
            "ok": False,
            "advice_text": "Recommendation database not available",
            "advice_html": "<div class='small-muted'>Please rest, stay hydrated, and consult a doctor.</div>",
            "urgent": False
        })

    print(f"Searching through {len(recommendations_data['diseases'])} diseases...")

    # Find matching recommendation
    recommendation = find_recommendation_json(disease)
    
    if recommendation:
        print(f"✅ MATCH FOUND: {recommendation.get('name')}")
        
        empathetic_msg = recommendation.get('empathetic_message', '')
        medications = recommendation.get('medications', '')
        home_remedies = recommendation.get('home_remedies', '')
        
        # Create HTML
        html = format_recommendation_html_from_json(recommendation)
        
        # Build speech text
        speech_parts = []
        if medications and 'consult' not in medications.lower():
            speech_parts.append(f"You can take {medications}")
        if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
            speech_parts.append(f"Try these home remedies: {home_remedies}")
        
        speech_text = ". ".join(speech_parts) if speech_parts else "Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        
        urgent = detect_emergency(medications + ' ' + home_remedies) or 'seek' in medications.lower() or 'consult' in medications.lower()
        
        print(f"✓ Returning recommendation")
        print(f"  Medications: {medications[:50]}...")
        print(f"  Home remedies: {home_remedies[:50]}...")
        print(f"  Urgent: {urgent}")
        
        return jsonify({
            "ok": True,
            "matched_disease": recommendation.get('name'),
            "advice_text": f"Medications: {medications}; Home remedies: {home_remedies}",
            "advice_html": html,
            "advice_text_for_speech": speech_text,
            "full_message": empathetic_msg,
            "urgent": urgent,
            "source": "json_database"
        })
    else:
        # Fallback
        print("❌ NO MATCH FOUND - Using fallback")
        print(f"  Searched for: '{disease}'")
        print(f"  Normalized: '{normalize_text(disease)}'")
        
        generic = "I'm sorry, I couldn't find specific recommendations for this condition. Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        return jsonify({
            "ok": True,
            "matched_disease": None,
            "advice_text": generic,
            "advice_html": f"<div class='small-muted'>{generic}</div>",
            "advice_text_for_speech": generic,
            "urgent": False,
            "source": "fallback"
        })

@app.route("/save_phone", methods=["POST"])
def save_phone():
    try:
        payload = request.get_json(silent=True) or {}
        phone = payload.get("phone", "").strip()
        name = payload.get("name", "").strip()  # Get name from payload
        disease = payload.get("disease", "Unknown")
        confidence = payload.get("confidence", 0)
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone required"}), 400
        
        if not name:
            return jsonify({"ok": False, "message": "Name required"}), 400
        
        digits = re.sub(r'[^\d]', '', phone)
        if not (10 <= len(digits) <= 14):
            return jsonify({"ok": False, "message": "Invalid phone format"}), 400
        
        # Check if file exists and has Name column
        file_exists = os.path.exists(PHONE_LOG_FILE)
        has_name_column = False
        
        if file_exists:
            with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
                first_line = f.readline().strip()
                has_name_column = 'Name' in first_line
        
        with open(PHONE_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # Write header if file is new
            if not file_exists:
                writer.writerow(['Timestamp', 'Name', 'Phone', 'Disease', 'Confidence', 'Status'])
                has_name_column = True
            
            # Write data row
            if has_name_column:
                writer.writerow([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    name,
                    phone,
                    disease,
                    confidence,
                    'Pending'
                ])
            else:
                # Old format without Name column - upgrade it
                writer.writerow([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    name,
                    phone,
                    disease,
                    confidence,
                    'Pending'
                ])
        
        print(f"✅ Saved patient: {name} ({phone}) - {disease}")
        
        return jsonify({
            "ok": True,
            "message": "Details saved successfully",
            "name": name,
            "phone": phone
        })
    except Exception as e:
        print(f"❌ Error saving patient: {e}")
        traceback.print_exc()
        return jsonify({"ok": False, "message": str(e)}), 500

# ============================================
# DOCTOR DASHBOARD ENDPOINTS
# ============================================

@app.route("/get_patients", methods=["GET"])
def get_patients():
    """
    Get all patients from the phone callbacks CSV file for doctor dashboard
    Handles both old format (without Name) and new format (with Name)
    """
    try:
        patients = []
        
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({
                "ok": True,
                "patients": [],
                "message": "No patients yet"
            })
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            fieldnames = reader.fieldnames
            
            # Check if Name column exists
            has_name_column = 'Name' in fieldnames if fieldnames else False
            
            print(f"📋 CSV columns: {fieldnames}")
            print(f"📋 Has Name column: {has_name_column}")
            
            for idx, row in enumerate(reader, 1):
                # Handle both old and new CSV formats
                if has_name_column:
                    name = row.get('Name', '').strip()
                    if not name:
                        name = f'Patient #{idx}'
                else:
                    # Old format without Name column - generate name from phone
                    phone = row.get('Phone', '')
                    name = f'Patient {phone[-4:]}' if phone and len(phone) >= 4 else f'Patient #{idx}'
                
                patients.append({
                    "timestamp": row.get('Timestamp', ''),
                    "name": name,
                    "phone": row.get('Phone', ''),
                    "disease": row.get('Disease', 'Unknown'),
                    "confidence": int(row.get('Confidence', 0)),
                    "status": row.get('Status', 'Pending')
                })
        
        # Sort by timestamp (most recent first)
        patients.sort(key=lambda x: x['timestamp'], reverse=True)
        
        print(f"📋 Returning {len(patients)} patients")
        
        return jsonify({
            "ok": True,
            "patients": patients,
            "total": len(patients)
        })
        
    except Exception as e:
        print(f"❌ Error getting patients: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e),
            "patients": []
        }), 500


@app.route("/update_patient_status", methods=["POST"])
def update_patient_status():
    """
    Update the status of a patient (e.g., Pending -> Called -> Completed)
    Handles CSV with or without Name column
    """
    try:
        data = request.get_json(silent=True) or {}
        phone = data.get('phone', '').strip()
        new_status = data.get('status', 'Called').strip()
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone number required"}), 400
        
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({"ok": False, "message": "No patient records found"}), 404
        
        # Read all rows
        rows = []
        updated = False
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            fieldnames = reader.fieldnames
            
            print(f"📋 CSV fieldnames: {fieldnames}")
            
            for row in reader:
                if row['Phone'] == phone:
                    row['Status'] = new_status
                    updated = True
                    print(f"✅ Updated {phone} to {new_status}")
                rows.append(row)
        
        if not updated:
            return jsonify({"ok": False, "message": "Patient not found"}), 404
        
        # Write back with same column structure
        with open(PHONE_LOG_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
        
        return jsonify({
            "ok": True,
            "message": f"Status updated to {new_status}",
            "phone": phone,
            "status": new_status
        })
        
    except Exception as e:
        print(f"❌ Error updating patient status: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500


@app.route("/get_patient_details/<phone>", methods=["GET"])
def get_patient_details(phone):
    """
    Get detailed information for a specific patient
    """
    try:
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({"ok": False, "message": "No patient records"}), 404
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row.get('Phone') == phone:
                    return jsonify({
                        "ok": True,
                        "patient": {
                            "timestamp": row.get('Timestamp', ''),
                            "name": row.get('Name', 'Unknown'),
                            "phone": row.get('Phone', ''),
                            "disease": row.get('Disease', 'Unknown'),
                            "confidence": int(row.get('Confidence', 0)),
                            "status": row.get('Status', 'Pending')
                        }
                    })
        
        return jsonify({
            "ok": False,
            "message": "Patient not found"
        }), 404
        
    except Exception as e:
        print(f"❌ Error getting patient details: {e}")
        traceback.print_exc()
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500


@app.route("/get_dashboard_stats", methods=["GET"])
def get_dashboard_stats():
    """
    Get statistics for doctor dashboard
    """
    try:
        if not os.path.exists(PHONE_LOG_FILE):
            return jsonify({
                "ok": True,
                "stats": {
                    "total": 0,
                    "pending": 0,
                    "called": 0,
                    "completed": 0,
                    "today": 0,
                    "urgent": 0
                }
            })
        
        total = 0
        pending = 0
        called = 0
        completed = 0
        today = 0
        urgent = 0
        
        today_date = datetime.now().date()
        
        with open(PHONE_LOG_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                total += 1
                status = row.get('Status', 'Pending')
                
                if status == 'Pending':
                    pending += 1
                elif status == 'Called':
                    called += 1
                elif status == 'Completed':
                    completed += 1
                
                # Check if today
                timestamp_str = row.get('Timestamp', '')
                if timestamp_str:
                    try:
                        record_date = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S').date()
                        if record_date == today_date:
                            today += 1
                    except:
                        pass
                
                # Check if urgent (low confidence or serious disease)
                confidence = int(row.get('Confidence', 100))
                disease = row.get('Disease', '').lower()
                urgent_keywords = ['pneumonia', 'heart', 'stroke', 'breathing', 'chest']
                
                if confidence < 50 or any(kw in disease for kw in urgent_keywords):
                    urgent += 1
        
        return jsonify({
            "ok": True,
            "stats": {
                "total": total,
                "pending": pending,
                "called": called,
                "completed": completed,
                "today": today,
                "urgent": urgent
            }
        })
        
    except Exception as e:
        print(f"Error getting dashboard stats: {e}")
        return jsonify({
            "ok": False,
            "error": str(e)
        }), 500


if __name__ == "__main__":
    print("\n" + "="*60)
    print("STARTUP CHECK")
    print("="*60)
    print(f"CSV loaded: {df is not None}")
    if df is not None:
        print(f"CSV shape: {df.shape}")
        print(f"CSV columns: {df.columns.tolist()}")
    print(f"Recommendations JSON loaded: {recommendations_data is not None}")
    if recommendations_data:
        print(f"Number of diseases in JSON: {len(recommendations_data.get('diseases', []))}")
        print(f"First 3 diseases: {[d.get('name') for d in recommendations_data.get('diseases', [])[:3]]}")
    print("="*60 + "\n")
    
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


LOADING RECOMMENDATIONS JSON
Looking for file: recommendations.json
Current directory: c:\Users\Jay\Desktop\SE proj\New folder\Back
File exists: True
✓ Successfully loaded JSON
  Number of diseases: 62
  First disease: Common Cold (Upper Respiratory Tract Infection)
  Sample aliases: ['common cold (upper respiratory tract infection)']

✓ Loaded CSV with encoding: utf-8
✓ CSV loaded successfully: 62 diseases

STARTUP CHECK
CSV loaded: True
CSV shape: (62, 6)
CSV columns: ['Sr No.', 'Disease', 'Typical Symptoms', 'Unique Symptom', 'Confirm Symptom', 'Recommendations']
Recommendations JSON loaded: True
Number of diseases in JSON: 62
First 3 diseases: ['Common Cold (Upper Respiratory Tract Infection)', 'Influenza (Flu)', 'COVID-19']

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
